In [1]:
import pandas as pd
import numpy as np

In [7]:
def calculate_calories(weight_kg, height_cm, age, gender, activity_level, goal):
    # BMR calculation
    if gender.lower() == 'male':
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
    else:
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161

    # Activity multiplier
    activity_multipliers = {
        'sedentary': 1.2,
        'light': 1.375,
        'moderate': 1.55,
        'active': 1.725,
        'very active': 1.9
    }
    activity_multiplier = activity_multipliers.get(activity_level.lower(), 1.2)

    tdee = bmr * activity_multiplier

    print("The tdee is :",tdee)

    # Goal adjustment
    goal = goal.lower()
    if goal == 'fatloss':
        return tdee - 500
    elif goal == 'musclegain':
        return tdee + 300
    elif goal == 'endurance':
        return tdee + 150
    else:  # maintenance
        return tdee

In [26]:
calculate_calories(74, 182, 23, "male", "moderate", "strength")

The tdee is : 2739.625


2739.625

In [8]:
MACRO_GUIDELINES = {
    "sedentary": {
        "fat_loss":     {"protein_g_per_kg": (1.2, 1.6), "fat_pct": 30, "carb_pct": 40},
        "muscle_gain":  {"protein_g_per_kg": (1.4, 1.8), "fat_pct": 25, "carb_pct": 50},
        "strength":     {"protein_g_per_kg": (1.4, 1.8), "fat_pct": 30, "carb_pct": 45},
        "endurance":    {"protein_g_per_kg": (1.2, 1.5), "fat_pct": 25, "carb_pct": 55}
    },
    "light": {
        "fat_loss":     {"protein_g_per_kg": (1.4, 1.8), "fat_pct": 30, "carb_pct": 40},
        "muscle_gain":  {"protein_g_per_kg": (1.6, 2.0), "fat_pct": 25, "carb_pct": 50},
        "strength":     {"protein_g_per_kg": (1.6, 2.0), "fat_pct": 25, "carb_pct": 50},
        "endurance":    {"protein_g_per_kg": (1.4, 1.6), "fat_pct": 25, "carb_pct": 55}
    },
    "moderate": {
        "fat_loss":     {"protein_g_per_kg": (1.6, 2.0), "fat_pct": 30, "carb_pct": 40},
        "muscle_gain":  {"protein_g_per_kg": (1.8, 2.2), "fat_pct": 25, "carb_pct": 50},
        "strength":     {"protein_g_per_kg": (1.8, 2.2), "fat_pct": 25, "carb_pct": 50},
        "endurance":    {"protein_g_per_kg": (1.5, 1.8), "fat_pct": 20, "carb_pct": 60}
    },
    "active": {
        "fat_loss":     {"protein_g_per_kg": (1.8, 2.2), "fat_pct": 30, "carb_pct": 40},
        "muscle_gain":  {"protein_g_per_kg": (2.0, 2.2), "fat_pct": 25, "carb_pct": 50},
        "strength":     {"protein_g_per_kg": (2.0, 2.2), "fat_pct": 25, "carb_pct": 50},
        "endurance":    {"protein_g_per_kg": (1.6, 1.8), "fat_pct": 20, "carb_pct": 60}
    },
    "very_active": {
        "fat_loss":     {"protein_g_per_kg": (2.0, 2.4), "fat_pct": 30, "carb_pct": 40},
        "muscle_gain":  {"protein_g_per_kg": (2.0, 2.4), "fat_pct": 20, "carb_pct": 55},
        "strength":     {"protein_g_per_kg": (2.0, 2.4), "fat_pct": 25, "carb_pct": 50},
        "endurance":    {"protein_g_per_kg": (1.6, 2.0), "fat_pct": 20, "carb_pct": 60}
    }
}

def calculate_macros(goal, activity_level, tdee, weight_kg, fitness_level):
    goal = goal.lower()
    activity_level = activity_level.lower()
    fitness_level = fitness_level.lower()

    guidelines = MACRO_GUIDELINES[activity_level][goal]
    
    # Step 1: Select protein per kg based on fitness level
    protein_range = guidelines["protein_g_per_kg"]
    if fitness_level == "beginner":
        protein_per_kg = protein_range[0]
    elif fitness_level == "intermediate":
        protein_per_kg = sum(protein_range) / 2
    elif fitness_level == "advanced":
        protein_per_kg = protein_range[1]
    else:
        raise ValueError("Invalid fitness_level")

    # Step 2: Calculate protein in grams and kcal
    protein_g = round(protein_per_kg * weight_kg)
    protein_kcal = protein_g * 4

    # Step 3: Calculate fat and carbs from remaining kcal
    fat_pct = guidelines["fat_pct"] / 100
    carb_pct = guidelines["carb_pct"] / 100
    total_pct = fat_pct + carb_pct

    remaining_kcal = tdee - protein_kcal
    fat_kcal = round((fat_pct / total_pct) * remaining_kcal)
    carb_kcal = round((carb_pct / total_pct) * remaining_kcal)

    fat_g = round(fat_kcal / 9)
    carb_g = round(carb_kcal / 4)

    return {
        "calories_kcal": tdee,
        "protein_g": protein_g,
        "fat_g": fat_g,
        "carbohydrate_g": carb_g,
        "protein_kcal": protein_kcal,
        "fat_kcal": fat_kcal,
        "carb_kcal": carb_kcal,
        "total_kcal_from_macros": protein_kcal + fat_kcal + carb_kcal
    }


In [15]:
calculate_macros("fat_loss", "moderate", 2050, 74, "beginner")

{'calories_kcal': 2050,
 'protein_g': 118,
 'fat_g': 75,
 'carbohydrate_g': 226,
 'protein_kcal': 472,
 'fat_kcal': 676,
 'carb_kcal': 902,
 'total_kcal_from_macros': 2050}

In [16]:
macros = calculate_macros(
    goal="muscle_gain",
    activity_level="moderate",
    tdee=2700,
    weight_kg=74,
    fitness_level="beginner"
)

print(macros)


{'calories_kcal': 2700, 'protein_g': 133, 'fat_g': 80, 'carbohydrate_g': 361, 'protein_kcal': 532, 'fat_kcal': 723, 'carb_kcal': 1445, 'total_kcal_from_macros': 2700}
